In [2]:
# ============================================================
# Cell 0 — Notebook purpose (Paper 1 scope)
# ============================================================
"""
SFH GW Table Generator (Paper 1: Scaling & Validation)

This notebook:
1) Computes timing-derived orbital decay proxy da/dt from (Pb, Pbdot).
2) Calibrates a single universal resistance scale gamma using ONE reference system:
   PSR J0737–3039.
3) Freezes gamma and computes out-of-sample SFH predictions for remaining systems.
4) Exports a reproducible CSV (and optional LaTeX) for Table 1.

Deliberately excluded in this notebook:
- Eccentricity enhancement factors f(e)
- PN corrections
- Waveform phasing
- Any multi-system fitting
"""

'\nSFH GW Table Generator (Paper 1: Scaling & Validation)\n\nThis notebook:\n1) Computes timing-derived orbital decay proxy da/dt from (Pb, Pbdot).\n2) Calibrates a single universal resistance scale gamma using ONE reference system:\n   PSR J0737–3039.\n3) Freezes gamma and computes out-of-sample SFH predictions for remaining systems.\n4) Exports a reproducible CSV (and optional LaTeX) for Table 1.\n\nDeliberately excluded in this notebook:\n- Eccentricity enhancement factors f(e)\n- PN corrections\n- Waveform phasing\n- Any multi-system fitting\n'

In [3]:
# ============================================================
# Cell 1 — Imports
# ============================================================
import math
from dataclasses import dataclass
from typing import Optional, Dict, List, Tuple

import numpy as np
import pandas as pd

In [4]:
# ============================================================
# Cell 2 — Physical constants + unit helpers (define ONCE)
# ============================================================
G = 6.67430e-11            # m^3 kg^-1 s^-2
c = 299_792_458.0          # m s^-1
M_sun = 1.98847e30         # kg

SEC_PER_DAY = 86400.0
SEC_PER_YEAR = 365.25 * SEC_PER_DAY

def to_mm_per_day(m_per_s: float) -> float:
    return m_per_s * SEC_PER_DAY * 1e3

def to_mm_per_year(m_per_s: float) -> float:
    return m_per_s * SEC_PER_YEAR * 1e3

In [5]:
# ============================================================
# Cell 3 — Observed inputs (ATNF Pulsar Catalogue)
# These values are observational and are NOT computed.
# ============================================================

# System: J0737–3039A (Double Pulsar)
# Source: ATNF Pulsar Catalogue v2.7.0

# Orbital period
# PB = 0.10225156248 days
#    = 8834.53 seconds

# Orbital period derivative
# PBDOT = 5.0e-11 (days/day)
#       = 5.0e-11 (s/s)

PB_J0737_sec = 8834.53
PBDOT_J0737 = -1.252e-12


In [6]:
# ============================================================
# Cell 3b — Data model (define ONCE)
# ============================================================

from dataclasses import dataclass

@dataclass(frozen=True)
class PulsarBinary:
    name: str
    Pb_s: float        # orbital period [seconds]
    Pbdot: float       # dP/dt [s/s]
    m1_Msun: float     # primary mass [solar masses]
    m2_Msun: float     # companion mass [solar masses]
    e: float           # eccentricity
    ref: str           # citation / note

In [7]:
# ============================================================
# Cell 4 — Input dataset (EDIT THIS)
# ============================================================
# IMPORTANT RULES:
# - Exactly one entry per physical pulsar system.
# - Use a single canonical name (e.g., keep B1913+16 OR J1915+1606, not both).
# - Do NOT enter observed da/dt by hand. Observed must be computed from Pb/Pbdot.

binaries: List[PulsarBinary] = [
    PulsarBinary(
        name="PSR J0737-3039",
        Pb_s=0.0,          # TODO: fill
        Pbdot=0.0,         # TODO: fill (negative)
        m1_Msun=0.0,       # TODO: fill
        m2_Msun=0.0,       # TODO: fill
        e=0.0,             # TODO: fill
        ref="TODO"
    ),
    PulsarBinary(
        name="PSR B1913+16",
        Pb_s=0.0,          # TODO
        Pbdot=0.0,         # TODO
        m1_Msun=0.0,
        m2_Msun=0.0,
        e=0.0,
        ref="TODO"
    ),
    # TODO: add the remaining systems...
]

CALIBRATOR_NAME = "PSR J0737-3039"

In [8]:
# ================================================
# Cell 4a — Observed orbital decay (unit conversion only)
# ================================================

PBDOT_J0737_sec_per_year = PBDOT_J0737 * SEC_PER_YEAR

print("Observed dPB/dt (s/year):", PBDOT_J0737_sec_per_year)

Observed dPB/dt (s/year): -3.95101152e-05


In [9]:
# ================================================
# Cell 4b — Convert observed dPB/dt to m/s
# ================================================

# a_dot / a = (2/3) * (Pdot / P)  → Keplerian relation
a_dot_over_a_obs = (2.0 / 3.0) * (PBDOT_J0737 / PB_J0737_sec)

# FREEZE the observable (do not reinterpret later)
a_dot_over_a_obs = -9.4478e-17  # 1/s  (observed, fixed)

print("Observed (a_dot / a) [1/s]:", a_dot_over_a_obs)

Observed (a_dot / a) [1/s]: -9.4478e-17


In [10]:
# ============================================================
# Cell 4c — SFH effective decay observable (definition only)
# ============================================================
# In SFH, the observable compared to timing data is not raw a_dot,
# but an effective decay proxy proportional to:
#
#   a_dot_eff ≡ C * a * (a_dot / a)
#
# where:
#   - a_dot_over_a_obs is the frozen observable from Cell 4b
#   - a is the semi-major axis
#   - C is a universal normalization fixed by ONE calibration system
#
# No numerical evaluation here.

# Placeholder symbol (do NOT compute yet)
a_dot_eff_obs = None

In [11]:
# ============================================================
# Cell 4d — Compute observed a_dot (m/s) from frozen a_dot_over_a_obs
# ============================================================

import math

# Masses for J0737–3039A/B (solar masses)
m1_J0737 = 1.3381
m2_J0737 = 1.2489
M_tot_J0737 = (m1_J0737 + m2_J0737) * M_sun

# Semi-major axis from Kepler: a^3 = G M (P / 2π)^2
a_J0737 = (G * M_tot_J0737 * (PB_J0737_sec / (2.0 * math.pi))**2)**(1.0/3.0)

# Observed a_dot (m/s)
a_dot_obs = a_dot_over_a_obs * a_J0737

print("a_J0737 (m):", a_J0737)
print("Observed a_dot (m/s):", a_dot_obs)

a_J0737 (m): 878839266.3003715
Observed a_dot (m/s): -8.30309762015265e-08


In [18]:
# ============================================================
# Cell 4e — Model raw prediction (placeholder)
# ============================================================
# This represents the SFH (or GR) raw prediction BEFORE calibration.
# For now, we set it equal to the observed a_dot as a placeholder.
# This will be replaced by the true model expression.

a_dot_eff_pred_raw = a_dot_obs  # placeholder, to be replaced

In [19]:
# ============================================================
# Cell 5 — Calibrate universal normalization C (J0737–3039 only)
# ============================================================

# Calibration is done ONCE, using the reference system only.
# We use absolute values to avoid sign-convention issues.

C = abs(a_dot_obs) / abs(a_dot_eff_pred_raw)

print("Calibrated C (using J0737–3039):", C)

Calibrated C (using J0737–3039): 1.0


In [21]:
# ============================================================
# Cell 6 — Calibrate gamma from J0737–3039 (Style A)
# ============================================================

# SFH leading-order quadrupole scaling (raw, before normalization):
# a_dot_eff ≈ (G^4 / c^5) * (m1*m2)^2 * (m1 + m2) / (gamma^2 * a^5)
#
# Solve for gamma using the observed decay for the calibration system.

m1 = m1_J0737 * M_sun
m2 = m2_J0737 * M_sun
a  = a_J0737

# Use absolute value to avoid sign conventions (decay direction handled separately)
gamma = math.sqrt(
    (G**4 / c**5) * (m1*m2)**2 * (m1 + m2) / (abs(a_dot_obs) * a**5)
)

print("Calibrated gamma (J0737–3039):", gamma)

Calibrated gamma (J0737–3039): 6502465318361101.0


In [22]:
# ============================================================
# Cell 7 — SFH prediction for PSR B1913+16 (out-of-sample)
# ============================================================

# --- Observed inputs for B1913+16 (ATNF / timing literature) ---
PB_B1913_sec   = 0.322997448918 * SEC_PER_DAY   # orbital period [s]
PBDOT_B1913    = -2.42e-12                      # intrinsic Pbdot [s/s]
e_B1913        = 0.6171334

m1_B1913 = 1.438 * M_sun
m2_B1913 = 1.390 * M_sun
Mtot_B1913 = m1_B1913 + m2_B1913

# --- Convert timing observable ---
a_dot_over_a_obs_B1913 = (2.0 / 3.0) * (PBDOT_B1913 / PB_B1913_sec)

# --- Semi-major axis from Kepler ---
a_B1913 = (G * Mtot_B1913 * (PB_B1913_sec / (2.0 * math.pi))**2)**(1.0/3.0)

# --- Observed decay proxy ---
a_dot_obs_B1913 = a_dot_over_a_obs_B1913 * a_B1913

# --- SFH raw prediction using frozen gamma ---
a_dot_pred_B1913 = (
    (G**4 / c**5)
    * (m1_B1913 * m2_B1913)**2
    * (Mtot_B1913)
    / (gamma**2 * a_B1913**5)
)

print("B1913+16:")
print("  a (m) =", a_B1913)
print("  observed a_dot (m/s) =", a_dot_obs_B1913)
print("  SFH predicted a_dot (m/s) =", a_dot_pred_B1913)
print("  ratio (pred / obs) =", a_dot_pred_B1913 / a_dot_obs_B1913)

B1913+16:
  a (m) = 1949051740.265416
  observed a_dot (m/s) = -1.1267683524163835e-07
  SFH predicted a_dot (m/s) = 2.4202957796540034e-09
  ratio (pred / obs) = -0.0214799765583016


In [23]:
# ============================================================
# Cell 8 — Eccentricity enhancement f(e) (GR-standard) + corrected ratio
# ============================================================

def f_eccentricity(e: float) -> float:
    # Peters & Mathews (1963): enhancement of GW power for eccentric binaries
    num = 1.0 + (73.0/24.0)*e**2 + (37.0/96.0)*e**4
    den = (1.0 - e**2)**(7.0/2.0)
    return num / den

f_B1913 = f_eccentricity(e_B1913)

a_dot_pred_B1913_ecc = a_dot_pred_B1913 * f_B1913

print("B1913+16 eccentricity correction:")
print("  e =", e_B1913)
print("  f(e) =", f_B1913)
print("  SFH predicted a_dot (ecc-corrected) (m/s) =", a_dot_pred_B1913_ecc)
print("  ratio (pred_ecc / obs) =", a_dot_pred_B1913_ecc / a_dot_obs_B1913)

B1913+16 eccentricity correction:
  e = 0.6171334
  f(e) = 11.85677382594044
  SFH predicted a_dot (ecc-corrected) (m/s) = 2.86968996512357e-08
  ratio (pred_ecc / obs) = -0.2546832238382846


In [24]:
# ============================================================
# Cell 9 — Calibrate gamma using PBDOT directly (J0737–3039 only)
# ============================================================

def f_eccentricity(e: float) -> float:
    num = 1.0 + (73.0/24.0)*e**2 + (37.0/96.0)*e**4
    den = (1.0 - e**2)**(7.0/2.0)
    return num / den

# --- J0737 inputs needed for PBDOT-calibration ---
# (e is small; include it for consistency)
e_J0737 = 0.0877775  # update later if you want exact ATNF value

m1 = m1_J0737 * M_sun
m2 = m2_J0737 * M_sun
Mtot = m1 + m2
Pb = PB_J0737_sec
a  = a_J0737

K = (G**4 / c**5) * (m1*m2)**2 * Mtot
fe = f_eccentricity(e_J0737)

# Mapping:  (a_dot/a) = (2/3)(Pbdot/Pb)  ⇒  Pbdot = (3/2) Pb (a_dot/a)
# SFH model (raw): a_dot = [K * f(e)] / (gamma^2 * a^5)
# ⇒ a_dot/a = [K * f(e)] / (gamma^2 * a^6)
# ⇒ Pbdot_pred = (3/2) Pb * [K * f(e)] / (gamma^2 * a^6)
# Solve for gamma using observed PBDOT_J0737:

gamma_pbdot = math.sqrt(
    (1.5 * Pb * K * fe) / (abs(PBDOT_J0737) * a**6)
)

print("J0737 PBDOT-based calibration:")
print("  e_J0737 =", e_J0737)
print("  f(e_J0737) =", fe)
print("  gamma_pbdot =", gamma_pbdot)

J0737 PBDOT-based calibration:
  e_J0737 = 0.0877775
  f(e_J0737) = 1.0515436227145931
  gamma_pbdot = 6667948348672679.0


In [26]:
# ============================================================
# Cell 10 — SFH prediction for PBDOT: PSR B1913+16 (out-of-sample)
# ============================================================

# --- B1913+16 inputs ---
Pb_B1913 = PB_B1913_sec
e_B1913  = e_B1913

m1 = m1_B1913
m2 = m2_B1913
Mtot = m1 + m2

a = a_B1913
K = (G**4 / c**5) * (m1*m2)**2 * Mtot
fe = f_eccentricity(e_B1913)

# SFH prediction for PBDOT using gamma_pbdot
PBDOT_pred_B1913 = (1.5 * Pb_B1913 * K * fe) / (gamma_pbdot**2 * a**6)

print("B1913+16 PBDOT comparison:")
print("  observed PBDOT =", PBDOT_B1913)
print("  SFH predicted PBDOT =", -PBDOT_pred_B1913)
print("  ratio (pred / obs) =", PBDOT_pred_B1913 / abs(PBDOT_B1913))

B1913+16 PBDOT comparison:
  observed PBDOT = -2.42e-12
  SFH predicted PBDOT = -5.861210778290032e-13
  ratio (pred / obs) = 0.24219879249132362


In [ ]:
# ============================================================
# Cell 11 — Multi-system PBDOT comparison table (Option A)
# ============================================================

rows = []

systems = [
    {
        "name": "PSR J0737–3039",
        "Pb_s": PB_J0737_sec,
        "Pbdot_obs": PBDOT_J0737,
        "m1": m1_J0737 * M_sun,
        "m2": m2_J0737 * M_sun,
        "e": e_J0737,
        "ref": "ATNF v2.7.0 (calibrator)"
    },
    {
        "name": "PSR B1913+16",
        "Pb_s": PB_B1913_sec,
        "Pbdot_obs": PBDOT_B1913,
        "m1": m1_B1913 * M_sun,
        "m2": m2_B1913 * M_sun,
        "e": e_B1913,
        "ref": "Hulse & Taylor"
    },
    # Add more systems HERE later (J1906+0746, B2127+11C, etc.)
]

for s in systems:
    m1, m2 = s["m1"], s["m2"]
    Mtot = m1 + m2

    a = semi_major_axis_m(s["Pb_s"], Mtot)
    fe = f_eccentricity(s["e"])

    K = (G**4 / c**5) * (m1 * m2)**2 * Mtot

    # SFH PBDOT prediction (same form as Cell 10)
    Pbdot_pred = -(1.5 * s["Pb_s"] * K * fe) / (gamma_pbdot**2 * a**6)

    ratio = Pbdot_pred / s["Pbdot_obs"]

    rows.append({
        "System": s["name"],
        "PBDOT_obs": s["Pbdot_obs"],
        "PBDOT_SFH": Pbdot_pred,
        "Ratio (SFH / Obs)": ratio,
        "e": s["e"],
        "f(e)": fe,
        "ref": s["ref"]
    })

df_pbdot = pd.DataFrame(rows)
df_pbdot

NameError: name 'semi_major_axis_m' is not defined

In [ ]:
# ============================================================
# Cell 12 — Canonical definition of gamma_pbdot (LOCKED)
# ============================================================

# gamma_pbdot is defined ONLY through the PBDOT observable
# It must NEVER be mixed with a_dot-based expressions

print("gamma_pbdot =", gamma_pbdot)
print("Definition: calibrated solely from J0737–3039 PBDOT")

gamma_pbdot = 6667948348672679.0
Definition: calibrated solely from J0737–3039 PBDOT


In [ ]:
# ============================================================
# Cell 13 — Rebuild the table (PBDOT-only, canonical, no mixing) [FINAL CLEAN]
# ============================================================

import numpy as np
import pandas as pd

# -----------------------------
# Hard guardrails (minimal)
# -----------------------------
if "gamma" in globals():
    print("WARNING: 'gamma' exists (a_dot-based). Ignoring it. Using gamma_pbdot ONLY.")

if "gamma_pbdot" not in globals():
    raise NameError("gamma_pbdot is not defined. Run the calibration cell first.")

# Sanity: require core constants to exist (from earlier cells)
_required = ["G", "c", "M_sun", "f_eccentricity"]
missing = [k for k in _required if k not in globals()]
if missing:
    raise NameError(f"Missing required globals {missing}. Run the constants / helper-function cells first.")

# -----------------------------
# Kepler semi-major axis (define locally to avoid NameError)
# -----------------------------
def semi_major_axis_m(Pb_s: float, Mtot_kg: float) -> float:
    """
    Semi-major axis from Kepler's third law.
    Pb_s: orbital period [s]
    Mtot_kg: total mass [kg]
    Returns: a [m]
    """
    return (G * Mtot_kg * Pb_s**2 / (4.0 * np.pi**2)) ** (1.0 / 3.0)

# -----------------------------
# Canonical SFH predictor (PBDOT-space ONLY)
# -----------------------------
def sfh_pbdot_pred(Pb_s: float, m1_Msun: float, m2_Msun: float, e: float) -> float:
    """
    Returns SFH predicted PBDOT (negative for orbital decay),
    using the canonical PBDOT-space formula and gamma_pbdot.
    """
    m1 = m1_Msun * M_sun
    m2 = m2_Msun * M_sun
    Mtot = m1 + m2

    a = semi_major_axis_m(Pb_s, Mtot)
    fe = f_eccentricity(e)

    K = (G**4 / c**5) * (m1 * m2)**2 * Mtot
    return -(1.5 * Pb_s * K * fe) / (gamma_pbdot**2 * a**6)

# -----------------------------
# EDIT THIS LIST ONLY
# (curated systems, one per physical binary)
# Add "is_cluster": True for globular cluster systems (dagger flag)
# -----------------------------
systems_curated = [
    {
        "System": "PSR J0737–3039",
        "Pb_s": PB_J0737_sec,
        "PBDOT_obs": PBDOT_J0737,
        "m1_Msun": m1_J0737,
        "m2_Msun": m2_J0737,
        "e": e_J0737,
        "ref": "ATNF v2.7.0 (calibrator)",
        "is_cluster": False,
    },
    {
        "System": "PSR B1913+16",
        "Pb_s": PB_B1913_sec,
        "PBDOT_obs": PBDOT_B1913,
        "m1_Msun": 1.438,
        "m2_Msun": 1.390,
        "e": e_B1913,
        "ref": "Timing literature (Hulse–Taylor)",
        "is_cluster": False,
    },
    {
        "System": "PSR J1906+0746",
        "Pb_s": 14342.400000000001,
        "PBDOT_obs": -8.3e-13,  # intrinsic (van Leeuwen et al. 2015)
        "m1_Msun": 1.29,
        "m2_Msun": 1.32,
        "e": 0.085,
        "ref": "van Leeuwen et al. (2015)",
        "is_cluster": False,
    },
    {
        "System": "PSR B2127+11C",
        "Pb_s": 27906.9808,
        "PBDOT_obs": -3.95e-12,  # observed (cluster acceleration contaminates intrinsic)
        "m1_Msun": 1.358,
        "m2_Msun": 1.354,
        "e": 0.681395,
        "ref": "Anderson et al. (1990); globular cluster system",
        "is_cluster": True,  # dagger
    },
    {
        "System": "PSR B1534+12",
        "Pb_s": 36352.447,
        "PBDOT_obs": -1.92e-13,
        "m1_Msun": 1.333,
        "m2_Msun": 1.345,
        "e": 0.2736775,
        "ref": "Stairs et al. (2002); high-precision timing",
        "is_cluster": False,
    },
]

# -----------------------------
# Build table
# -----------------------------
rows = []
for s in systems_curated:
    Pb_s = float(s["Pb_s"])
    Pbdot_obs = float(s["PBDOT_obs"])
    m1 = float(s["m1_Msun"])
    m2 = float(s["m2_Msun"])
    e = float(s["e"])

    Mtot_kg = (m1 + m2) * M_sun
    a_m = semi_major_axis_m(Pb_s, Mtot_kg)
    fe = f_eccentricity(e)

    Pbdot_pred = sfh_pbdot_pred(Pb_s, m1, m2, e)
    ratio = Pbdot_pred / Pbdot_obs

    sys_label = s["System"] + (r"$^{\dagger}$" if s.get("is_cluster", False) else "")

    rows.append({
        "System": sys_label,
        "System_raw": s["System"],
        "Type": ("observed" if s.get("is_cluster", False) else "intrinsic"),
        "PBDOT_obs": Pbdot_obs,
        "PBDOT_SFH": Pbdot_pred,
        "Ratio (SFH/Obs)": ratio,
        "e": e,
        "f(e)": fe,
        "Pb_s": Pb_s,
        "a_m": a_m,
        "m1_Msun": m1,
        "m2_Msun": m2,
        "ref": s.get("ref", ""),
    })

df_table = pd.DataFrame(rows)

# Compact, paper-facing view
display(df_table[["System", "Type", "PBDOT_obs", "PBDOT_SFH", "Ratio (SFH/Obs)", "e", "f(e)", "ref"]])

print("\nFootnote:")
print(r"  $\dagger$ Observed value; intrinsic $\dot P_b$ uncertain due to globular-cluster acceleration.")

print("\nSummary:")
print("  gamma_pbdot =", gamma_pbdot)
print("  N systems =", len(df_table))

,System,Type,PBDOT_obs,PBDOT_SFH,Ratio (SFH/Obs),e,f(e),ref
0,PSR J0737–3039,intrinsic,-1.252000e-12,-1.252000e-12,1.000000,0.087777,1.051544,ATNF v2.7.0 (calibrator)
1,PSR B1913+16,intrinsic,-2.420000e-12,-5.861211e-13,0.242199,0.617133,11.856774,Timing literature (Hulse–Taylor)
2,PSR J1906+0746,intrinsic,-8.300000e-13,-3.001815e-13,0.361665,0.085000,1.048266,van Leeuwen et al. (2015)
3,PSR B2127+11C$^{\dagger}$,observed,-3.950000e-12,-9.673855e-13,0.244908,0.681395,22.176838,Anderson et al. (1990); globular cluster system
4,PSR B1534+12,intrinsic,-1.920000e-13,-3.069175e-14,0.159853,0.273678,1.615240,Stairs et al. (2002); high-precision timing



Footnote:
  $\dagger$ Observed value; intrinsic $\dot P_b$ uncertain due to globular-cluster acceleration.

Summary:
  gamma_pbdot = 6667948348672679.0
  N systems = 5


In [ ]:
# ============================================================
# Cell 14 — Export table (CSV + LaTeX) for paper use
# ============================================================

# Choose the paper-facing columns and format
df_paper = df_table[[
    "System", "PBDOT_obs", "PBDOT_SFH", "Ratio (SFH/Obs)", "e", "f(e)", "ref"
]].copy()

# Optional: nicer scientific formatting (strings) for LaTeX
def fmt_sci(x, sig=3):
    return f"{x:.{sig}e}"

df_tex = df_paper.copy()
df_tex["PBDOT_obs"] = df_tex["PBDOT_obs"].map(lambda v: fmt_sci(v, 3))
df_tex["PBDOT_SFH"] = df_tex["PBDOT_SFH"].map(lambda v: fmt_sci(v, 3))
df_tex["Ratio (SFH/Obs)"] = df_tex["Ratio (SFH/Obs)"].map(lambda v: f"{v:.3f}")
df_tex["e"] = df_tex["e"].map(lambda v: f"{v:.6f}")
df_tex["f(e)"] = df_tex["f(e)"].map(lambda v: f"{v:.3f}")

# Save CSV (machine-readable)
csv_path = "sfh_pbdot_table_optionA.csv"
df_paper.to_csv(csv_path, index=False)

# Produce LaTeX (paper-ready)
latex_str = df_tex.to_latex(
    index=False,
    escape=False,
    column_format="lrrrrrl",
    caption="Binary pulsar orbital decay comparison in timing observable $\\dot P_b$. "
            "A single universal resistance scale $\\gamma$ is calibrated on PSR~J0737--3039 "
            "and applied unchanged to other systems. The table tests leading-order scaling.",
    label="tab:sfh_pbdot_comparison"
)

print("Wrote:", csv_path)
print("\nLaTeX table:\n")
print(latex_str)

Wrote: sfh_pbdot_table_optionA.csv

LaTeX table:

\begin{table}
\caption{Binary pulsar orbital decay comparison in timing observable $\dot P_b$. A single universal resistance scale $\gamma$ is calibrated on PSR~J0737--3039 and applied unchanged to other systems. The table tests leading-order scaling.}
\label{tab:sfh_pbdot_comparison}
\begin{tabular}{lrrrrrl}
\toprule
System & PBDOT_obs & PBDOT_SFH & Ratio (SFH/Obs) & e & f(e) & ref \\
\midrule
PSR J0737–3039 & -1.252e-12 & -1.252e-12 & 1.000 & 0.087777 & 1.052 & ATNF v2.7.0 (calibrator) \\
PSR B1913+16 & -2.420e-12 & -5.861e-13 & 0.242 & 0.617133 & 11.857 & Timing literature (Hulse–Taylor) \\
PSR J1906+0746 & -4.000e-13 & -3.002e-13 & 0.750 & 0.085000 & 1.048 & ATNF / timing paper (fill exact values) \\
PSR B2127+11C & -3.950e-12 & -9.674e-13 & 0.245 & 0.681395 & 22.177 & Anderson et al. 1990; globular cluster system \\
PSR B1534+12 & -1.920e-13 & -3.069e-14 & 0.160 & 0.273678 & 1.615 & Stairs et al. 2002; high-precision timing \\
\bot

In [ ]:
{
    "name": "PSR J1906+0746",
    "Pb_s": 0.166 * SEC_PER_DAY,      # TODO: replace with exact PB from ATNF
    "Pbdot_obs": -4.0e-13,            # TODO: replace with intrinsic PBDOT from literature/ATNF
    "m1_Msun": 1.29,                  # TODO: replace with best-estimate masses used in timing paper
    "m2_Msun": 1.32,                  # TODO
    "e": 0.085,                       # TODO: replace with exact e
    "ref": "ATNF / timing paper (fill exact values)"
},

({'name': 'PSR J1906+0746',
  'Pb_s': 14342.400000000001,
  'Pbdot_obs': -4e-13,
  'm1_Msun': 1.29,
  'm2_Msun': 1.32,
  'e': 0.085,
  'ref': 'ATNF / timing paper (fill exact values)'},)

In [ ]:
# ============================================================
# Cell 9 — DEFINE SFH prediction formula (EDIT THIS)
# ============================================================
"""
You must paste your Paper-1 SFH model here.

Requirements:
- It must take (m1, m2, a, G, c, gamma) and return adot_SFH in m/s.
- No eccentricity factor (Paper 1).
- No fitted coefficients inside this function.
- The only adjustable quantity is gamma, calibrated once.

Template below uses a placeholder 'F_scale(...)'. Replace with your actual formula.
"""

def adot_sfh_m_per_s(m1_kg: float, m2_kg: float, a_m: float, gamma: float) -> float:
    # TODO: Replace the body with your Paper-1 equation.
    # Example skeleton:
    # M = m1_kg + m2_kg
    # mu = (m1_kg*m2_kg)/M
    # return -(1.0/gamma**2) * (G**4/c**5) * (mu**2 * M**3) / (a_m**3)  # <-- NOT REAL; placeholder
    raise NotImplementedError("Paste your SFH adot formula here.")

In [ ]:
# ============================================================
# Cell 10 — Calibrate gamma using the single reference system
# ============================================================
def find_binary(name: str) -> PulsarBinary:
    for b in binaries:
        if b.name == name:
            return b
    raise KeyError(name)

cal = find_binary(CALIBRATOR_NAME)
a_cal = semi_major_axis_m(cal.Pb_s, cal.M_kg)
adot_cal_obs = adot_from_timing_m_per_s(a_cal, cal.Pb_s, cal.Pbdot)

print("Calibrator:", cal.name)
print("a_cal (m):", a_cal)
print("Observed adot (mm/day):", to_mm_per_day(adot_cal_obs))

In [ ]:
# ============================================================
# Cell 11 — Solve for gamma (1D root-find)
# ============================================================
def solve_gamma_for_calibrator(
    target_adot_m_per_s: float,
    m1_kg: float,
    m2_kg: float,
    a_m: float,
    gamma_lo: float = 1e60,
    gamma_hi: float = 1e90,
    max_iter: int = 200
) -> float:
    """
    Assumes adot_sfh ∝ 1/gamma^2 (monotonic in gamma for fixed system).
    Uses bisection on f(gamma) = adot_sfh(gamma) - target.
    """
    def f(g):
        return adot_sfh_m_per_s(m1_kg, m2_kg, a_m, g) - target_adot_m_per_s

    flo = f(gamma_lo)
    fhi = f(gamma_hi)

    if np.sign(flo) == np.sign(fhi):
        raise ValueError(
            "Gamma bracket does not straddle the target. "
            "Adjust gamma_lo/gamma_hi or check adot_sfh_m_per_s sign/scale."
        )

    lo, hi = gamma_lo, gamma_hi
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fmid = f(mid)
        if abs(fmid) < 1e-30:  # extremely tight; adjust if needed
            return mid
        if np.sign(fmid) == np.sign(flo):
            lo, flo = mid, fmid
        else:
            hi, fhi = mid, fmid
    return 0.5 * (lo + hi)

# Once you implement adot_sfh_m_per_s, this will produce gamma.
gamma = solve_gamma_for_calibrator(
    target_adot_m_per_s=adot_cal_obs,
    m1_kg=cal.m1_kg,
    m2_kg=cal.m2_kg,
    a_m=a_cal,
)

print("Calibrated gamma:", gamma)

In [ ]:
# ============================================================
# Cell 12 — Derived bookkeeping constant A_GW (optional; derived only)
# ============================================================
# Only compute A_GW from gamma if you want it for reporting.
# Note: With A_GW = (1/gamma^2)*(c^5/G^3), A_GW has units of s^2.

A_GW = (1.0 / gamma**2) * (c**5 / (G**3))
print("Derived A_GW:", A_GW, " [units: s^2 under this definition]")

In [ ]:
# ============================================================
# Cell 13 — Compute SFH predictions for all systems (gamma frozen)
# ============================================================
df_pred = df.copy()

adot_sfh_list = []
for _, r in df_pred.iterrows():
    b = find_binary(r["name"])
    adot_sfh = adot_sfh_m_per_s(b.m1_kg, b.m2_kg, r["a_m"], gamma)
    adot_sfh_list.append(adot_sfh)

df_pred["adot_sfh_m_per_s"] = adot_sfh_list
df_pred["adot_sfh_mm_per_day"] = df_pred["adot_sfh_m_per_s"].map(to_mm_per_day)

df_pred["ratio_sfh_over_obs"] = df_pred["adot_sfh_m_per_s"] / df_pred["adot_obs_m_per_s"]
df_pred["resid_mm_per_day"] = df_pred["adot_sfh_mm_per_day"] - df_pred["adot_obs_mm_per_day"]

df_pred[["name", "adot_obs_mm_per_day", "adot_sfh_mm_per_day", "ratio_sfh_over_obs", "e"]]

In [ ]:
# ============================================================
# Cell 14 — Mandatory sanity checks (stop if these fail)
# ============================================================
# 1) Calibrator must match (within numerical tolerance)
cal_row = df_pred[df_pred["name"] == CALIBRATOR_NAME].iloc[0]
if abs(cal_row["ratio_sfh_over_obs"] - 1.0) > 1e-6:
    raise AssertionError("Calibrator does not match exactly. Check solver and formula.")

# 2) No NaNs
if df_pred.isna().any().any():
    raise AssertionError("NaNs found in prediction table.")

print("Sanity checks passed.")

In [ ]:
# ============================================================
# Cell 15 — Export CSV + optional LaTeX table
# ============================================================
OUT_CSV = "sfh_gw_table1_reproducible.csv"
df_out = df_pred[[
    "name", "m1_Msun", "m2_Msun", "Pb_s", "Pbdot", "e", "a_m",
    "adot_obs_mm_per_day", "adot_sfh_mm_per_day",
    "ratio_sfh_over_obs", "resid_mm_per_day", "ref"
]].copy()

df_out.to_csv(OUT_CSV, index=False)
print("Wrote:", OUT_CSV)

# Optional: LaTeX table snippet (edit formatting as needed)
latex = df_out.to_latex(index=False, float_format=lambda x: f"{x:.4g}")
with open("sfh_gw_table1.tex", "w") as f:
    f.write(latex)
print("Wrote: sfh_gw_table1.tex")

In [ ]:
# ============================================================
# Cell 16 — Print the "paper-ready" summary lines
# ============================================================
print("=== Calibration summary ===")
print("Calibrator:", CALIBRATOR_NAME)
print("gamma =", gamma)
print("A_GW (derived, s^2) =", A_GW)
print("\n=== Table preview ===")
display(df_out.head(10))

In [ ]:
# ============================================================
# Cell 20 - 6b — Dataset validation (out-of-sample systems only)
# ============================================================

names = [b.name for b in binaries]

# Ensure no duplicate physical systems
if len(names) != len(set(names)):
    dupes = pd.Series(names)[pd.Series(names).duplicated()].unique().tolist()
    raise ValueError(f"Duplicate system names detected: {dupes}")

# Validate ONLY out-of-sample systems (skip calibrator)
for b in binaries:
    if b.name == CALIBRATOR_NAME:
        continue  # calibrator handled separately

    if b.Pb_s <= 0:
        raise ValueError(f"Pb_s not set for {b.name}")
    if b.Pbdot == 0:
        raise ValueError(f"Pbdot not set for {b.name}")

print("✅ Out-of-sample dataset validation passed.")

ValueError: Pb_s not set for PSR B1913+16